### 풀이
- (자력) 어떤 수 $A, B$를 합치는 것($AB$라고 표현)은 $A * 10^{|B|} + B$와 같다.
  - $\scriptsize |B|$는 $B$의 자릿수를 의미한다.
- $A$와 $B$를 합친것을 $AB$라고 할 때, $AB \mod k = (A * 10^{|B|} + B) \mod k = (A * 10^{|B|} \mod k + B \mod k) \mod k$이다.
  - 수식은 모듈러 연산의 분배법칙에 의해 유도된 것이다.
  - 따라서 숫자를 직접 합칠 필요 없이, 두 수의 나머지를 이용한 연산으로 나온 모든 값도, $k$가 100이하이므로, 모두 저장할 수 있다. (이 문제풀이의 핵심)
- `DP[i][j]` = 숫자를 합친 것이 `i` 상태일 때, 나머지가 j인 순열의 개수
  - `DP[현재 상태][합친 나머지] += DP[이전상태][이전 나머지]`

#### 순열에 대해
- DP의 상태를 나타내는 bit field가 (그것을 고르는)순열이 아닌 조합이 아닌가에 대한 의혹을 해결하기 위해 자세히 고찰해보았다.
- 예를들어 DP의 상태가 4bit라고 할 때, 0011은 0010과 0001에서 넘어온다.
  - 즉 0010에서 4번째 숫자를 추가한 경우와 0001에서 3번째 숫자를 추가한 경우와 같다고 볼 수 있다.
  - 따라서 0011은 이전 상태에서 순서를 고려했으므로 순열이라고 할 수 있다.
- 0111의 경우도 0011, 0101, 0110 에서 넘어오고, 이 과정에서 추가될 때 순서가 고려되므로 순열이라고 볼 수 있다.

In [ ]:
import math

N = int(input())
L = [int(input()) for _ in range(N)]
K = int(input())
DP = [[-1] * K for _ in range(1 << N)] # DP[bit][mod]
DP[0][0] = 1 # 0개를 골라서 나머지가 0인 경우는 1가지

def len(n) :
  return int(math.log10(int(n))) + 1

def p2(n) :
  return int(math.log2(n))

def sub(n):
  len = n.bit_length()
  for i in range(len):
    if n & (1 << i):
      sub = n & ~(1 << i)
      yield sub

def dp(bit, mod):
  if DP[bit][mod] != -1:
    return DP[bit][mod]

  DP[bit][mod] = 0
  for s in sub(bit): # s는 bit의 부분집합
    x = p2(bit^s)
    b = L[x]
    for j in range(K) :
      m = ((j * (10 ** len(b))) % K + b % K) % K
      DP[bit][m] += dp(s, j)
  return DP[bit][mod]
dp((1 << N) - 1, 0)

- 뭔가 이해를 잘못한 코드

In [ ]:
import math

def sol():
  N = int(input())
  L = [int(input()) for _ in range(N)]
  K = int(input())
  DP = [[0] * K for _ in range(1 << N)] # DP[bit][mod]
  DP[0][0] = 1 # 0개를 골라서 나머지가 0인 경우는 1가지

  LK = [l % K for l in L] # 각 숫자의 나머지를 미리 구해놓음
  tenK = [(10 ** n) % K for n in range(51)] # n은 최대 50자리, 10^n % K를 미리 구해놓음
  length = [len(str(l)) for l in L] # 각 숫자의 자릿수를 미리 구해놓음

  for i in range(1 << N):
    for j in range(K) :
      for l in range(N):
        if i & (1 << l) : continue
        DP[i | (1 << l)][((j * tenK[length[l]]) % K + LK[l] % K) % K] += DP[i][j] #합쳐서 나온 나머지를 축적시킴
  
  p = DP[i | (1 << l)][0]
  q = math.perm(N)
  g = math.gcd(p, q)
  print(f'{p//g}/{q//g}')

sol()

### 풀이2
- Bottom-up 방식 풀이
  - 템플릿으로 기억할 필요가 있을진 모르겠다.
- i상태에 있는 나머지들의 조합중, j번 loop돌면서, 합쳐서 나온 나머지(`((j * tenK[length[l]]) % K + LK[l] % K) % K`)를 기존에 있던 값에서 추가시킨다.
- DP안에 있는 값의 의미는 참 비직관적이다.. 
  - 사실상 기저 상태와 최종상태를 제외하면 너무나도 복잡한 의미를 가지고 있는 것 같다.

In [ ]:
import math, sys
input=sys.stdin.readline

def sol():
  N = int(input())
  L = [int(input()) for _ in range(N)]
  K = int(input())
  DP = [[0] * K for _ in range(1 << N)] # DP[bit][mod]
  DP[0][0] = 1 # 0개를 골라서 나머지가 0인 경우는 1가지

  R = [[(j * 10 ** len(str(L[i])) + L[i]) % K for j in range(K)] for i in range(N)] #j뒤에 i번째 숫자를 합친 것을 K로 나눈 나머지를 저장
  for cur, l in enumerate(DP):
    for i, ll in enumerate(R):
      if cur & (1 << i): continue
      for j in range(K):
        DP[cur | (1 << i)][ll[j]] += l[j]
  
  p = DP[(1 << N) - 1][0]
  q = sum(DP[(1 << N) - 1])
  g = math.gcd(p, q)
  print(f'{p//g}/{q//g}')

sol()

### 풀이 3
- i번째 숫자를 0부터 k까지의 숫자들로 합친 모든 조합에 대한 나머지를 구해놓은 `r`을 기반으로 푼다.
  - `r[i][j]` = j뒤에 i번째 숫자를 합친 숫자를 K로 나눈 나머지
  - 즉, L[i]로부터 나뉘어질 수 있는 모든 경우의 수를 구해놓은 것이다.
- Bottom-up을 탐색하는 방법: `dp[b|(1<<i)]` = (i번째 수를 포함시킨 상태) = (다음 상태) 
  - `b` = (현재 상태)